Visualize the precipitation events next to each other

In [ ]:
import pandas as pd
import numpy as np

import os

from load_wastewater_data import *
from plotting_tools import *
import seaborn as sns

# Update rcParams to set the default font to Times New Roman
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'DejaVu Serif'

In [ ]:
plot_path = f"../plots/wastewater_model/"
os.makedirs(os.path.join(plot_path), exist_ok=True)
os.makedirs(os.path.join(plot_path, "rain_scenarios"), exist_ok=True)

# Overview of Rain Settings

In [ ]:
# load data
df_gentle = pd.read_csv(os.path.join("../..", "parameters_and_models", "wastewater_model", "nieselregen.txt"), sep="\t")
df_moderate = pd.read_csv(os.path.join("../..", "parameters_and_models", "wastewater_model", "mittelstarker_regen.txt"), sep="\t")

df_gentle["rain_scenario"] = "Nieselregen"
df_moderate["rain_scenario"] = "MittelstarkerRegen"
df = pd.concat([df_gentle, df_moderate])


df.columns = ["time", "l/s*ha", "rain_scenario"]
df.time = df.time / 60 / 24 # minutes -> days

### Plot moderate precipiation events next to each other

In [ ]:
fig, axs = plt.subplots(ncols=3, sharey=True, figsize=(15.5, 3), dpi=300)
for i, (variate, variate_title) in enumerate(zip(["Nieselregen", "MittelstarkerRegen"], ["moderate gentle rain showers", "moderate rain showers"])):
    sns.lineplot(data=df.loc[df.rain_scenario==variate,:], x="time", y="l/s*ha", ax=axs[i], color=["cornflowerblue", "navy"][i])
    axs[i].set_title(f"{variate_title}")    
    axs[i].set_xlabel(f"simulation time [days]")
    axs[i].set_ylim(0,140)  # Set y-limit to 0
    if i==0:
        axs[i].set_ylabel(f"precipitation\n[l/s⋅ha]")
    else:
        axs[i].set_ylabel(" ")
    axs[i].tick_params(axis='x')
    axs[i].tick_params(axis='y')

plt.tight_layout()
fig.savefig(os.path.join(plot_path, "rain_scenarios", f"precipitation_overview_share_y.png"))

### Experiment with second form of visualization

In [ ]:
df["hours"] = df.time * 24

aggregated_hours = 2

bins = pd.interval_range(start=0, end=336, freq=aggregated_hours)

# Use pd.cut to assign each value to a bin
df['bin'] = pd.cut(df['hours'], bins)

# Group by the bins and sum the values
aggregated_df = df.groupby('bin')['l/s*ha'].sum().reset_index()

aggregated_df[f"days"] = np.linspace(0, 14, len(bins))

In [ ]:
aggregated_df.head()

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(aggregated_df[f'days'], aggregated_df['l/s*ha'], width=1/24*aggregated_hours, edgecolor='black')

# Add labels and title
plt.xlabel('simulation time (days)')
plt.ylabel('precipitation (l/s*ha)')
plt.title('Histogram of Precipitation over Time')